<a href="https://colab.research.google.com/github/hammad93/hurricane-net/blob/main/hurricane_net_chatgpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.2 MB/s eta 0:00:00


In [2]:
# set OpenAI API key to environment variable
import os
os.environ["OPENAI_API_KEY"] = ''

In [5]:
import requests
# we import the class like this
# todo in the future we will be able to import from a pypi package
data = requests.get('https://raw.githubusercontent.com/hammad93/hurricane-net/main/hurricane_net_chatgpt.py').text
exec(data)

In [6]:
live_storms = get_live_storms()
live_storms

,id,time,lat,lon,wind_speed,wind_speed_mph,wind_speed_kph
0,WP82023,2023-08-19 16:56:00,20.1,157.0,15,27.78,17.2617
1,WP82023,2023-08-19 22:56:00,20.0,156.6,15,27.78,17.2617
2,WP82023,2023-08-20 04:56:00,19.9,156.2,15,27.78,17.2617
3,WP82023,2023-08-20 10:56:00,19.8,155.8,15,27.78,17.2617
4,WP82023,2023-08-20 16:56:00,19.6,155.7,15,27.78,17.2617
...,...,...,...,...,...,...,...
126,AL922023,2023-08-23 16:56:00,16.8,-35.8,20,37.04,23.0156
127,AL922023,2023-08-23 22:56:00,17.2,-36.4,20,37.04,23.0156
128,AL922023,2023-08-24 04:56:00,17.4,-37.4,20,37.04,23.0156
129,AL922023,2023-08-24 10:56:00,17.6,-38.6,20,37.04,23.0156


In [ ]:
forecasts = chatgpt_reflection_forecast_concurrent()

[
  {
    "time":"2023-08-24 22:56:00",
    "lat":17.0,
    "lon":152.6,
    "wind_speed":30
  },
  {
    "time":"2023-08-24 16:56:00",
    "lat":16.5,
    "lon":151.3,
    "wind_speed":30
  },
  {
    "time":"2023-08-24 10:56:00",
    "lat":16.3,
    "lon":150.3,
    "wind_speed":30
  },
  {
    "time":"2023-08-24 04:56:00",
    "lat":16.4,
    "lon":149.8,
    "wind_speed":25
  },
  {
    "time":"2023-08-23 22:56:00",
    "lat":16.6,
    "lon":149.7,
    "wind_speed":25
  },
  {
    "time":"2023-08-23 16:56:00",
    "lat":16.8,
    "lon":150.1,
    "wind_speed":25
  },
  {
    "time":"2023-08-23 10:56:00",
    "lat":17.2,
    "lon":150.7,
    "wind_speed":25
  },
  {
    "time":"2023-08-23 04:56:00",
    "lat":17.1,
    "lon":151.3,
    "wind_speed":25
  },
  {
    "time":"2023-08-22 22:56:00",
    "lat":17.6,
    "lon":151.4,
    "wind_speed":20
  },
  {
    "time":"2023-08-22 16:56:00",
    "lat":18.2,
    "lon":151.9,
    "wind_speed":20
  },
  {
    "time":"2023-08-22 10:56:00",


In [ ]:
forecasts

[[{'lat': 19.6,
   'lon': 125.3,
   'wind_speed': 25,
   'id': 'WP92023',
   'time': datetime.datetime(2023, 8, 24, 0, 0),
   'metadata': {'forecast_hour': 6, 'model': 'gpt-3.5-turbo'}},
  {'lat': 19.7,
   'lon': 125.0,
   'wind_speed': 25,
   'id': 'WP92023',
   'time': datetime.datetime(2023, 8, 24, 6, 0),
   'metadata': {'forecast_hour': 12, 'model': 'gpt-3.5-turbo'}},
  {'lat': 19.8,
   'lon': 124.9,
   'wind_speed': 20,
   'id': 'WP92023',
   'time': datetime.datetime(2023, 8, 24, 18, 0),
   'metadata': {'forecast_hour': 24, 'model': 'gpt-3.5-turbo'}},
  {'lat': 26.2,
   'lon': 112.2,
   'wind_speed': 25,
   'id': 'WP92023',
   'time': datetime.datetime(2023, 8, 25, 18, 0),
   'metadata': {'forecast_hour': 48, 'model': 'gpt-3.5-turbo'}},
  {'lat': 19.7,
   'lon': 125.5,
   'wind_speed': 30,
   'id': 'WP92023',
   'time': datetime.datetime(2023, 8, 26, 18, 0),
   'metadata': {'forecast_hour': 72, 'model': 'gpt-3.5-turbo'}},
  {'lat': 20.1,
   'lon': 123.9,
   'wind_speed': 25,
   '

In [ ]:
import openai
model_version = "gpt-3.5-turbo"
example_messages = [
  {"role": "system", "content": "Please act like an expert in tropical storm forecasting. Responses will be used for research and not for official purposes. Responses should be based on historical data and forecasts must be as accurate as possible."},
  {"role": "user", "content": example_prompt},
]
response = openai.ChatCompletion.create(
        model=model_version,
        messages=example_messages
        )
text = response["choices"][0]["message"]["content"]
print(text)

Based on the historical data provided, the forecast for 12 hours in the future from the most recent time is as follows:

{
  "id": "EP72023",
  "time": "2023-08-13T12:00:00",
  "lat": 15.2,
  "lon": -114.6,
  "wind_speed": 40
}


In [ ]:
reflection_message = example_messages + [
  {"role": "assistant", "content": text},
  {"role": "user", "content": example_reflection}]
response = openai.ChatCompletion.create(
        model=model_version,
        messages=reflection_message
        )
text = response["choices"][0]["message"]["content"]
print(text)

False, the forecast provided is not appropriate as it simply repeats the most recent entry from the historical data. The forecast should be based on the historical data but adjusted to reflect the predicted conditions 12 hours in the future. I apologize for the oversight.

A revised forecast for 12 hours in the future from the most recent time, based on the historical data, would be as follows:

{
  "id": "EP72023",
  "time": "2023-08-13T12:00:00",
  "lat": 15.4,
  "lon": -115.4,
  "wind_speed": 45
}


<OpenAIObject chat.completion id=chatcmpl-7mvvCsLE0FBgOzy8pB4uCjlakJ3j7 at 0x7c69d34dc630> JSON: {
  "id": "chatcmpl-7mvvCsLE0FBgOzy8pB4uCjlakJ3j7",
  "object": "chat.completion",
  "created": 1691896750,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"forecasts\": [\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T00:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -102.5,\n      \"wind_speed\": 25\n    },\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T06:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -104.0,\n      \"wind_speed\": 25\n    },\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T12:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -105.2,\n      \"wind_speed\": 25\n    },\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T18:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -106.0,\n      \"wind_speed\": 25\n    },\n    